In [20]:

%matplotlib inline
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from urllib.request import urlopen 

plt.style.use('ggplot')
pd.set_option('display.max_columns', 500) 

In [5]:
breast_cancer = pd.read_csv("/home/jasp/Documents/python/data.csv")

In [6]:
breast_cancer.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [7]:
breast_cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
id                         569 non-null int64
diagnosis                  569 non-null object
radius_mean                569 non-null float64
texture_mean               569 non-null float64
perimeter_mean             569 non-null float64
area_mean                  569 non-null float64
smoothness_mean            569 non-null float64
compactness_mean           569 non-null float64
concavity_mean             569 non-null float64
concave points_mean        569 non-null float64
symmetry_mean              569 non-null float64
fractal_dimension_mean     569 non-null float64
radius_se                  569 non-null float64
texture_se                 569 non-null float64
perimeter_se               569 non-null float64
area_se                    569 non-null float64
smoothness_se              569 non-null float64
compactness_se             569 non-null float64
concavity_se               569 non

In [8]:
breast_cancer['id']

0        842302
1        842517
2      84300903
3      84348301
4      84358402
5        843786
6        844359
7      84458202
8        844981
9      84501001
10       845636
11     84610002
12       846226
13       846381
14     84667401
15     84799002
16       848406
17     84862001
18       849014
19      8510426
20      8510653
21      8510824
22      8511133
23       851509
24       852552
25       852631
26       852763
27       852781
28       852973
29       853201
         ...   
539      921362
540      921385
541      921386
542      921644
543      922296
544      922297
545      922576
546      922577
547      922840
548      923169
549      923465
550      923748
551      923780
552      924084
553      924342
554      924632
555      924934
556      924964
557      925236
558      925277
559      925291
560      925292
561      925311
562      925622
563      926125
564      926424
565      926682
566      926954
567      927241
568       92751
Name: id, Length: 569, d

In [9]:
breast_cancer.set_index(['id'],inplace = True)

In [10]:
breast_cancer['diagnosis'] = breast_cancer['diagnosis'].map({'M':1,'B':0})

In [11]:
breast_cancer.apply(lambda x:x.isnull().sum())

diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed: 32                569
dtype: i

In [12]:
breast_cancer = breast_cancer.drop(['Unnamed: 32'],axis = 1)

In [13]:
print("la dimension de notre data frame est la suivante:\n",breast_cancer.shape)

la dimension de notre data frame est la suivante:
 (569, 31)


In [14]:
print("le type de chaque colonne de notre data frame:\n",breast_cancer.dtypes)

le type de chaque colonne de notre data frame:
 diagnosis                    int64
radius_mean                float64
texture_mean               float64
perimeter_mean             float64
area_mean                  float64
smoothness_mean            float64
compactness_mean           float64
concavity_mean             float64
concave points_mean        float64
symmetry_mean              float64
fractal_dimension_mean     float64
radius_se                  float64
texture_se                 float64
perimeter_se               float64
area_se                    float64
smoothness_se              float64
compactness_se             float64
concavity_se               float64
concave points_se          float64
symmetry_se                float64
fractal_dimension_se       float64
radius_worst               float64
texture_worst              float64
perimeter_worst            float64
area_worst                 float64
smoothness_worst           float64
compactness_worst          float64
concavi

In [15]:

def F(df,col):
    col_val = df[col].value_counts()
    col_val = col_val.reset_index()
    f = lambda x,y:100*(x/sum(y))
    for i in range(0,len(col_val['index'])):
         print(' on a {1:.2f}% pour le label:',col_val['index'][i],f(col_val[col].iloc[i],col_val[col]))
    
F(breast_cancer,'diagnosis')

 on a {1:.2f}% pour le label: 0 62.741652021089635
 on a {1:.2f}% pour le label: 1 37.258347978910365


In [16]:
col_val = breast_cancer['diagnosis'].value_counts()
col_val = col_val.reset_index()
col_val['index']

0    0
1    1
Name: index, dtype: int64

In [17]:
X = breast_cancer.iloc[:,breast_cancer.columns!='diagnosis']
y = breast_cancer.iloc[:,breast_cancer.columns=='diagnosis']

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [19]:
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [21]:
fit_gb = GradientBoostingClassifier(random_state = 42)

In [26]:
np.random.seed(42)
start = time.time()
parameters = {'loss':["deviance"],
              'learning_rate':[0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
     "max_depth":[2,3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
}

In [27]:
cv_gb = GridSearchCV(fit_gb,parameters, cv = 10, n_jobs=-1)

In [28]:
cv_gb.fit(X_train,y_train)

KeyboardInterrupt: 

In [ ]:
print(cv_gb.score(X_train,y_train))
print(cv_gb.best_params_)